In [ ]:
# processes landslide data

In [1]:
import os
import sys
import math
import warnings
import yaml
import pandas as pd
import geopandas as gpd
from osgeo import gdal
from pathlib import Path
import glob
import numpy as np
import rasterio.mask
import rasterio
from rasterio.plot import show
import fiona

In [2]:
aoi_folder = Path('data/AOI')
city_list = pd.read_csv('centroids.csv').city
output_folder = Path('data')

In [3]:
def clipdata_ls(city):
    city_no_space = city.replace(" ", "_")
    city_lower = city_no_space.lower()
    file = city / aoi_folder / (city_lower + '_AOI.shp')
    
    with fiona.open(file, "r") as shapefile:
        features = [feature["geometry"] for feature in shapefile]
        
        # Raw data folder. Change file path as needed
        input_raster = r'F:\World Bank\City Scan\data\Landslide\Global landslide hazard map - Rainfall trigger (1980-2018, median) - COG.tif'
        with rasterio.open(input_raster) as src:
            # shapely presumes all operations on two or more features exist in the same Cartesian plane.
            out_image, out_transform = rasterio.mask.mask(
                src, features, crop=True)
            out_meta = src.meta.copy()

        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform,
                         'nodata': 0})
        
        if np.nansum(out_image) != 0:
            output_file = city_lower + '_landslide.tif'
            with rasterio.open(Path(city) / output_folder / output_file, "w", **out_meta) as dest:
                dest.write(out_image)

In [4]:
for city in city_list:
    clipdata_ls(city)
# note that cities where landslide hazard is absent will not have a landslide raster file